In [1]:
import bq_helper
from bq_helper import BigQueryHelper
import pickle

import pandas as pd
import numpy as np
import datetime

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/xupingwang/key/google-bq-key.json"

pd.options.display.max_columns = 80

# Google BigQuery API set up, download 2016-2017 data

In [2]:
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
google_analytics = bq_helper.BigQueryHelper(\
                active_project="bigquery-public-data",\
                dataset_name="data:google_analytics_sample")

In [3]:
query = """
SELECT
visitNumber,
visitId,
visitStartTime,
date,
fullVisitorId,
channelGrouping,
totals,
trafficSource,
device,
geoNetwork
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
;
"""
df = google_analytics.query_to_pandas_safe(query)

In [56]:
# with open('googleyear.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('googleyear.pkl', 'rb') as f:
    df = pickle.load(f)

## User behavioral data(hits column) download

In [ ]:
# note that "hit" column is Json nested with list, 
# we will download the action_type field seperately  
query2 = """SELECT
fullVisitorId,
visitStartTime,
hits.eCommerceAction.action_type
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, UNNEST(hits) as h
WHERE
_TABLE_SUFFIX BETWEEN '20160801' AND '20160801'
;
"""
df_hits = google_analytics.query_to_pandas_safe(query2)

In [54]:
# with open('googlehits.pkl', 'wb') as f:
#     pickle.dump(df_hits, f)

with open('googlehits.pkl', 'rb') as f:
    df_hits = pickle.load(f)

# Merge dataframes

In [55]:
# combine visitor_ID and visitor_start_time to produce unique id for each visit
df_hits["ID"] = df_hits["fullVisitorId"].astype(str) + \
df_hits["visitStartTime"].astype(str)

# action_type explained:
# Click through of product lists = 1, 
# Product detail views = 2, 
# Add product(s) to cart = 3, 
# Remove product(s) from cart = 4, 
# Check out = 5, 
# Completed purchase = 6, 
# Refund of purchase = 7, 
# Checkout options = 8, 
# Unknown = 0.
# seems like Checkout option

# for each visit, groupby id and find maximum action type(how close is a visitor to purchase an item)
df_hits = df_hits[["ID", "action_type"]].groupby("ID")["action_type"].apply(list)
df_hits = df_hits.to_frame()
df_hits["ID"] = df_hits.index

# check for the visitors that clicks into the checkout page, 1 for True and 0 for False
df_hits["action_type"] = df_hits.action_type.apply(lambda x: 1 if "5" in x else 0)

In [60]:
df["ID"] = df["fullVisitorId"].astype(str) + df["visitStartTime"].astype(str)
df_hits = df_hits.reset_index(drop=True)
df = pd.merge(df, df_hits, left_on='ID', right_on="ID")

In [64]:
# with open('google_full.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('google_full.pkl', 'rb') as f:
    df = pickle.load(f)